### Image preprocessing for CNN Model

In [83]:
import cv2
import matplotlib.pyplot as plt
from pdf2image import convert_from_path
import numpy as np

In [84]:
def removeLine(image):
    #image = cv2.imread(path)
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # Remove horizontal
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25,1))
    detected_lines = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
    cnts = cv2.findContours(detected_lines, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        cv2.drawContours(image, [c], -1, (255,255,255), 2)

    # Repair image
    repair_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,6))
    result = 255 - cv2.morphologyEx(255 - image, cv2.MORPH_CLOSE, repair_kernel, iterations=1)
    
    global i
    print('Reached :',i)
    # plt.imshow(thresh)
    # plt.imshow(detected_lines)
    # plt.imshow(image)
    cv2.imwrite('data/preprocess/'+str(i)+'.png',result)
    i+=1
    return result

In [85]:
def boundingBoxes(contours, ary):
    c_info = []
    for c in contours:
        x,y,w,h = cv2.boundingRect(c)
        c_im = np.zeros(ary.shape)
        cv2.drawContours(c_im, [c], 0, 255, -1)
        c_info.append({
            'x1': x,
            'y1': y,
            'x2': x + w - 1,
            'y2': y + h - 1,
#             'sum': np.sum(ary * (c_im > 0))/255
        })
    return c_info

In [95]:
def segmenter(im):
    im = np.array(im)
    
    copy = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(copy, 100, 200)
    
    rfkernel = np.ones((25,25),np.float32)/625
    rf = cv2.filter2D(edges,-1,rfkernel)
    
    ret,thresh1 = cv2.threshold(rf,48,255,cv2.THRESH_BINARY)    
    dkernel = np.ones((5,5), np.uint8)
#     erode = cv2.erode(thresh1, dkernel, iterations = 1)
    dialate = cv2.dilate(thresh1, dkernel, iterations=5)
    
    contours, hierarchy = cv2.findContours(dialate, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    boxes = boundingBoxes(contours, dialate)
    rect = im
    for b in boxes:
        x1,y1,x2,y2 = b['x1'],b['y1'],b['x2'],b['y2']
        rect = cv2.rectangle(rect,(x1,y1),(x2,y2),(0,0,255),1)
    
    cv2.imwrite('data/preprocess/edge-'+str(i)+'.png',edges)
    cv2.imwrite('data/preprocess/rf-'+str(i)+'.png',rf)
    cv2.imwrite('data/preprocess/dia-'+str(i)+'.png',dialate)
#     cv2.imwrite('data/preprocess/ero-'+str(i)+'.png',erode)
    cv2.imwrite('data/preprocess/thresh-'+str(i)+'.png',thresh1)
    cv2.imwrite('data/preprocess/rect-'+str(i)+'.png',rect)
    removeLine(im)

In [96]:
def pdfToImage(path):
    images = convert_from_path(path)
    segmenter(images[0])
#     for i in images:
#         segmenter(i)

In [97]:
i = 1
im = pdfToImage('data/convert.pdf')

Reached : 1
